# Mastering LinkedIn API: Step-by-Step Guide for Seamless Integration

LinkedIn offers a powerful API that allows developers to automate posting and retrieve user information. In this guide, we will walk you through the process of integrating with LinkedIn’s API, covering GET requests, posting text, uploading images, and posting videos.

## Step 1: Setting Up LinkedIn API Access

To begin, follow these steps to set up your LinkedIn API access:

1. **Ensure you have a LinkedIn account** – If you don’t have one, sign up at [LinkedIn](https://www.linkedin.com).
2. **Create a Company Page** – You need a company page to post on behalf of an organization.
3. **Register an App** – Go to [LinkedIn Developer Portal](https://developer.linkedin.com/) and create an app.
4. **Use Your Page Name** – When creating an app, use the name of your LinkedIn Page.
5. **Enable Products** – Under the **Products** tab, add:
   - **Share on LinkedIn**
   - **Sign In with LinkedIn using OpenID Connect**
6. **Verify LinkedIn Page** – In the **Settings** tab, verify your LinkedIn page to allow posting access.
7. **Generate Access Token** – Navigate to the **Auth** tab and use the **OAuth 2.0 tools** on the right side to generate an access token.
8. **Get User Asset ID using Postman** – Once you obtain the access token, use **Postman** to make API requests and retrieve the asset ID.

---

## Step 2: Making a GET Request to Retrieve User Info

To fetch user details, use the following Python script:
- Loads environment variables to securely store your **LinkedIn Access Token**.
- Makes a GET request to LinkedIn’s API to fetch user details.
- If successful, prints user information; otherwise, displays an error.


In [5]:
import requests
import os
from dotenv import load_dotenv  # ✅ Import load_dotenv

load_dotenv()
ACCESS_TOKEN = os.getenv("LINKEDIN_ACCESS_TOKEN")
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}

response = requests.get("https://api.linkedin.com/v2/userinfo", headers=headers)

if response.status_code == 200:
    print("✅ User Info:", response.json())
else:
    print(f"❌ Error: {response.status_code}, {response.text}")


✅ User Info: {'sub': 'DLIaV4SiSh', 'email_verified': True, 'name': 'MAYANK GUPTA', 'locale': {'country': 'US', 'language': 'en'}, 'given_name': 'MAYANK', 'family_name': 'GUPTA', 'email': 'mayank.guptacse1@gmail.com'}


## Step 3: Posting Text to LinkedIn


Once you have your **User URN**, you can post a text update:
- Defines the API endpoint for posting text.
- Constructs the payload including the text message and visibility settings.
- Sends a POST request with the payload to LinkedIn.
- Handles success or failure responses.

In [ ]:
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get LinkedIn Access Token
ACCESS_TOKEN = os.getenv("LINKEDIN_ACCESS_TOKEN")
USER_PROFILE = os.getenv("PROFILE_URN")
USER_URN = "urn:li:person:USER_PROFILE"  # Replace with your LinkedIn Profile URN

# API Endpoint
url = "https://api.linkedin.com/v2/ugcPosts"

# Request Headers
headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "X-Restli-Protocol-Version": "2.0.0",
    "Content-Type": "application/json"
}

# Post Payload
payload = {
    "author": USER_URN,
    "lifecycleState": "PUBLISHED",
    "specificContent": {
        "com.linkedin.ugc.ShareContent": {
            "shareCommentary": {
                "text": "🚀 Automating LinkedIn posts using Python API! #Automation #LinkedInAPI"
            },
            "shareMediaCategory": "NONE"
        }
    },
    "visibility": {
        "com.linkedin.ugc.MemberNetworkVisibility": "PUBLIC"
    }
}

# Send the POST request
response = requests.post(url, headers=headers, json=payload)

# Handle response
if response.status_code == 201:
    print("✅ Successfully posted on LinkedIn!")
else:
    print(f"❌ Error: {response.status_code}, {response.text}")


✅ Successfully posted on LinkedIn!


## Step 4: Uploading an Image

### Step 4.1: Register Image Upload

- Registers an image upload by sending a request to LinkedIn.
- Retrieves an **Asset URN** and **Upload URL** for uploading the image.

Step-by-Step Breakdown

1️⃣ **API Endpoint Used**

https://api.linkedin.com/v2/assets?action=registerUpload
This endpoint tells LinkedIn that you are about to upload an image.

2️⃣ **Headers Explanation**

"Authorization": f"Bearer {ACCESS_TOKEN}" → Uses your LinkedIn access token. 

"X-Restli-Protocol-Version": "2.0.0" → Required for LinkedIn API v2. 

"Content-Type": "application/json" → JSON format request.

In [27]:
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

ACCESS_TOKEN = os.getenv("LINKEDIN_ACCESS_TOKEN")
USER_PROFILE = os.getenv("PROFILE_URN")

USER_URN = f"urn:li:person:{USER_PROFILE}"  # Replace with your LinkedIn Profile URN

# API endpoint to register an image upload
register_url = "https://api.linkedin.com/v2/assets?action=registerUpload"

headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "X-Restli-Protocol-Version": "2.0.0",
    "Content-Type": "application/json"
}

# Payload to request an upload URL
register_payload = {
    "registerUploadRequest": {
        "recipes": ["urn:li:digitalmediaRecipe:feedshare-image"],
        "owner": USER_URN,
        "serviceRelationships": [
            {"relationshipType": "OWNER", "identifier": "urn:li:userGeneratedContent"}
        ]
    }
}

# Send request to register the image
response = requests.post(register_url, headers=headers, json=register_payload)

if response.status_code == 200:
    data = response.json()
    asset_urn = data["value"]["asset"]
    upload_url = data["value"]["uploadMechanism"]["com.linkedin.digitalmedia.uploading.MediaUploadHttpRequest"]["uploadUrl"]
    print(f"✅ Image Upload Registered! Asset URN: {asset_urn}")
else:
    print(f"❌ Error: {response.status_code}, {response.text}")
    exit()


✅ Image Upload Registered! Asset URN: urn:li:digitalmediaAsset:D4D22AQEzpCQ7sxKYXw


### Step 4.2: Upload the Image

Once we have an upload URL, we use it to upload the image.

Upload the Image to LinkedIn

In [28]:
image_path = "../data/upload_image/AI-Pitch-Analysis.png"  # Replace with your image file path

# Open and upload the image
with open(image_path, "rb") as image_file:
    upload_response = requests.put(upload_url, data=image_file, headers={"Authorization": f"Bearer {ACCESS_TOKEN}"})

if upload_response.status_code == 201:
    print("✅ Image Uploaded Successfully!")
else:
    print(f"❌ Upload Error: {upload_response.status_code}, {upload_response.text}")
    exit()


✅ Image Uploaded Successfully!



### Step 4.3: Share Image Post
Create the LinkedIn Post with the Image

Now that the image is uploaded, we can use its asset URN in our post. Post with an Image!!!

In [29]:
# API endpoint to create a post
post_url = "https://api.linkedin.com/v2/ugcPosts"

# Payload to create a LinkedIn post with the image
post_payload = {
    "author": USER_URN,
    "lifecycleState": "PUBLISHED",
    "specificContent": {
        "com.linkedin.ugc.ShareContent": {
            "shareCommentary": {
                "text": "🚀 Automating LinkedIn second image posts with Python API! #Automation #LinkedInAPI"
            },
            "shareMediaCategory": "IMAGE",
            "media": [
                {
                    "status": "READY",
                    "description": {"text": "LinkedIn API Image Upload"},
                    "media": asset_urn,  # Use the asset URN from Step 1
                    "title": {"text": "Automation Success!"}
                }
            ]
        }
    },
    "visibility": {
        "com.linkedin.ugc.MemberNetworkVisibility": "PUBLIC"
    }
}

# Send the post request
post_response = requests.post(post_url, headers=headers, json=post_payload)

if post_response.status_code == 201:
    print("✅ Successfully posted on LinkedIn with an image!")
else:
    print(f"❌ Post Error: {post_response.status_code}, {post_response.text}")


✅ Successfully posted on LinkedIn with an image!


## Step 5: Uploading a Video

### Step 5.1: Register Video Upload
(Similar to Image Upload, but use `urn:li:digitalmediaRecipe:feedshare-video` in the payload.)

In [ ]:
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
ACCESS_TOKEN = os.getenv("LINKEDIN_ACCESS_TOKEN")
USER_PROFILE = os.getenv("PROFILE_URN") # Replace with your LinkedIn Member ID

# API Endpoint for registering video upload
register_url = "https://api.linkedin.com/v2/assets?action=registerUpload"

headers = {
    "Authorization": f"Bearer {ACCESS_TOKEN}",
    "X-Restli-Protocol-Version": "2.0.0",
    "Content-Type": "application/json"
}

# Payload for video upload registration
register_payload = {
    "registerUploadRequest": {
        "recipes": ["urn:li:digitalmediaRecipe:feedshare-video"],
        "owner": USER_URN,
        "serviceRelationships": [
            {"relationshipType": "OWNER", "identifier": "urn:li:userGeneratedContent"}
        ]
    }
}

# Sending the request
response = requests.post(register_url, headers=headers, json=register_payload)

if response.status_code == 200:
    data = response.json()
    asset_urn = data["value"]["asset"]  # This is the unique video asset identifier
    upload_url = data["value"]["uploadMechanism"]["com.linkedin.digitalmedia.uploading.MediaUploadHttpRequest"]["uploadUrl"]
    print(f"✅ Video Upload Registered! Asset URN: {asset_urn}")
    print(f"⬆️ Upload URL: {upload_url}")
else:
    print(f"❌ Error: {response.status_code}, {response.text}")
    exit()


✅ Video Upload Registered! Asset URN: urn:li:digitalmediaAsset:D4D05AQHlB-kjHcQ-yA
⬆️ Upload URL: https://www.linkedin.com/dms-uploads/sp/v2/D4D05AQHlB-kjHcQ-yA/uploadedVideo/B4DZXiYsqYG8AI-/0?ca=vector_feedshare&cn=uploads_secure&ccn=ambry-video&iri=B01-77&sync=0&v=beta&ut=0Eqa9q6hRXlrI1


### Step 5.2: Upload the Video
(Use the upload URL to send the video data.)
Now that we have the upload_url, we can upload the video via an HTTP PUT request.

In [31]:
import requests

video_path = "../data/upload_video/RAG Over Audio.mp4"  # Replace with the actual video file path

# Open the video file in binary mode
with open(video_path, "rb") as video_file:
    video_data = video_file.read()

# Send the video file to LinkedIn
upload_response = requests.put(upload_url, headers={"Authorization": f"Bearer {ACCESS_TOKEN}"}, data=video_data)

if upload_response.status_code == 201:
    print("✅ Video uploaded successfully!")
else:
    print(f"❌ Upload failed: {upload_response.status_code}, {upload_response.text}")


✅ Video uploaded successfully!


### Step 5.3: Share Video Post
(Use the asset URN to create a video post.)
Once the video is uploaded, use the asset_urn to create a post with the video.

In [32]:
# API Endpoint to create a post
post_url = "https://api.linkedin.com/v2/ugcPosts"

post_payload = {
    "author": USER_URN,  # Your LinkedIn Member URN
    "lifecycleState": "PUBLISHED",
    "specificContent": {
        "com.linkedin.ugc.ShareContent": {
            "shareCommentary": {
                "text": "Check out this awesome video!"
            },
            "shareMediaCategory": "VIDEO",
            "media": [
                {
                    "status": "READY",
                    "description": {
                        "text": "This is my uploaded video."
                    },
                    "media": asset_urn,  # Video URN from Step 1
                    "title": {
                        "text": "My Video Post"
                    }
                }
            ]
        }
    },
    "visibility": {
        "com.linkedin.ugc.MemberNetworkVisibility": "PUBLIC"
    }
}

# Send the request to publish the post
post_response = requests.post(post_url, headers=headers, json=post_payload)

if post_response.status_code == 201:
    print("✅ Video Post Published!")
else:
    print(f"❌ Post failed: {post_response.status_code}, {post_response.text}")


✅ Video Post Published!


## Conclusion
This guide walks you through LinkedIn API integration for retrieving user details, posting text, and sharing images and videos. 🚀 Master these steps to automate your LinkedIn workflow seamlessly!

### Blog Link - https://dev.to/mayankcse/mastering-linkedin-api-step-by-step-guide-for-seamless-integration-124n
